# How to use the FormationEnergyQE Workchain

This notebook will explain the use of the `FormationEnergyQE` workchain. 

This workchain implements the computation of the formation energy of a defect using QuantumESPRESSO.
This is one typical use-case of `AiiDA-Defects` and might be used in the situation in which you already have a defect structure you wish to work with. Note that there is also a `FormationEnergyBase` workchain, but this workchain cannot be used directly. One should always used one of the workchains dervived for a particular simulation code, such as `FormationEnergyQE`.

In example below, the `GaussianCountercharge` correction workchain is used as the electrostatic correction. After specifying some details about the defect of interest and making some QuantumESPRESSO specific choices, the `FormationEnergyQE` workchain will complete the following steps:
1. Run self-consistent DFT energy calculations for the host, neutral defect and charged defect supercells.
2. Run PP.x to obtain the electrostatic potentials.
3. Run a further DFT calculation on a unitcell (optionally).
4. Run DFPT to obtain the relative permitivitty.
5. Run the `GaussianCountercharge` workchain to obtain the correction.
6. Compute the formation energy, with and without corrections.

**NOTE!**
In this alpha version of `AiiDA-Defects` there are a number of features which are either not yet implemented or not well tested. These are listed below, with no guarentee of this being an exhaustive list.
Please bear these considerations in mind when testing the workchain.

* The PP.x plugin used must be from [my AiiDA-QuantumESPRESSO fork](https://github.com/ConradJohnston/aiida-quantumespresso/tree/pp-parser) until this is merged into the official release
* Alignment of the electrostatic potentials is not yet automatic. Placeholder code will return 0.0 eV for these steps. The alignment should be done 'by hand'. A typical option is to take a planar average of the electrostatic potential along some axis, and then to align far from the defect. For cubic cells, a convenient option is to place point defects in the center of the box, and then take the alignment at the edge. 
* In principle, only cubic cells are currently supported, although for a large-enough supercell, it shouldn't matter. This would be interesting to prove/disprove. A change to any shape of supercell is on the TODO list.
* The width of the model gaussian is fixed currently, with a TODO for a routine to fit this. If one wants to play with this for testing, I can expose it as an input.




## Step-by-step Usage 
1. Set up AiiDA as normal

In [ ]:
# Get your normal profile
from aiida import load_profile
load_profile()

# Import commonly used functionality
import numpy as np
from aiida import orm, engine, common
from aiida.plugins import WorkflowFactory

2. Import the workchain. Any issues with the import, such as the entry point not being found, would indicate an installation problem.

In [ ]:
from aiida_defects.formation_energy.formation_energy_qe import FormationEnergyWorkchainQE

3. Prepare some structures

In [ ]:
# Set up structures
import pymatgen

pymatgen_structure = pymatgen.Structure.from_file("./Structures/Diamond_1x1x1.cif")

# Unitcell
unitcell_structure = orm.StructureData(pymatgen=pymatgen_structure)

# Host 2x2x2 supercell
pymatgen_structure.make_supercell([2,2,2])
host_structure = orm.StructureData(pymatgen=pymatgen_structure)

# Defect (Carbon vacancy) 2x2x2 supercell
pymatgen_structure.remove_sites(indices=[0])
defect_structure = orm.StructureData(pymatgen=pymatgen_structure)

4. Prepare the inputs for the workchain. A builder can be used (as per the `GaussianCountercharge` example) or the input dictionary can be populated directly as shown below.

4a.  Set up the supercell calculations

In [ ]:
# Set up a PW calculation

# PW code
pw_code = orm.Code.get_from_string('pw@localhost')

# Add the minimum parameters needed, plus any additional needed for the system of interest
pw_parameters = orm.Dict(dict={
          'CONTROL': {
              'calculation': 'scf',
              'restart_mode': 'from_scratch',
              'wf_collect': True,
              },
          'SYSTEM': {
              'ecutwfc': 45,
              'ecutrho': 360.,
              },
          'ELECTRONS': {
              'conv_thr': 1.e-7,
              }})

# Setup k-points
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly

# Psuedos 
pseudo_family = 'SSSP' # This is the label that was used when installing the pseudos
pseudos = orm.nodes.data.upf.get_pseudos_from_structure(host_structure,pseudo_family)

# Scheduler options
pw_metadata = orm.Dict( dict={
    'description': 'Diamond test', 
    'options': {
        'max_wallclock_seconds': 1800, 
        'resources': {
            'num_machines': 1
        }
    }, 
    'label': 'Diamond test'
})


4b. Set up the unitcell calculation. This is optional, but likely to be necessary to converge the relative permitivitty. Using a unitcell, but with a much denser k-mesh is the expected usage.

In [ ]:
# Set up the unitcell PW calculation 
kpoints_unitcell = orm.KpointsData()
kpoints_unitcell.set_kpoints_mesh([20,20,20])

4c. Set up the post-processing calculations. These are used to obtain the electrostatic potentials from the supercell calculations.

In [ ]:
pp_code = orm.Code.get_from_string('pp@localhost')

# Scheduler options
pp_metadata = orm.Dict( dict={
    'description': 'Diamond test', 
    'options': {
        'max_wallclock_seconds': 1800, 
        'resources': {
            'num_machines': 1
        }
    }, 
    'label': 'Diamond test'
})


4d. Set up the post-processing calculations. These are used to obtain the electrostatic potentials from the supercell calculations.

In [ ]:
ph_code = orm.Code.get_from_string('ph@localhost')


ph_metadata = orm.Dict( dict={
    'description': 'Diamond test', 
    'options': {
        'max_wallclock_seconds': 1800, 
        'resources': {
            'num_machines': 28
        },
        'queue_name' : 'debug'
    }, 
    'label': 'Diamond test'
})

5. Construct the input dictionary. Note that this nesting is unique to the QuantumESPRESSO flavour of the FormationEnergy workchain.

In [ ]:
inputs = {
    # Structures
    'host_structure': host_structure,
    'defect_structure': defect_structure,
    'host_unitcell' : unitcell_structure,
    # Defect information 
    'defect_charge' : orm.Float(-2.),  
    'defect_site' : orm.List(list=[0.,0.,0.]),    # Position of the defect in crystal coordinates
    'fermi_level' : orm.Float(0.0),               # Position of the Fermi level, with respect to the valence band maximum      
    'chemical_potential' : orm.Float(250.709), # eV, the chemical potentical of a C atom
    # Method
    'correction_scheme' : orm.Str('gaussian'),
    # Computational (chosen code is QE)
    'qe' : {
        'dft': {
            'supercell' : {
                'code' : pw_code,
                'kpoints': kpoints, 
                'pseudopotentials': pseudos, 
                'parameters' : pw_parameters,
                'scheduler_options' : pw_metadata,

            },
            'unitcell' : {
                'code' : pw_code,
                'kpoints': kpoints_unitcell, 
                'pseudopotentials': pseudos, 
                'parameters' : pw_parameters,
                'scheduler_options' : pw_metadata,
            }            
        },
        'dfpt' : {
            'code' : ph_code,
            'scheduler_options' : ph_metadata,
            
        },
        'pp' : {
            'code' : pp_code,
            'scheduler_options' : pw_metadata,
        }
    }
}

6. Submit the workchain

In [ ]:
workchain_future = engine.submit(FormationEnergyWorkchainQE, **inputs)

7. Check the output

In [ ]:
if workchain_future.is_finished:
    if workchain_future.is_finished_ok:
        print('Finished successfully')
        print(
            "Workchain Formation energy (uncorrected): {} eV".format(
                workchain_future.outputs.formation_energy_uncorrected.value
            )
        )
        print(
            "Workchain Formation energy (corrected): {} eV".format(
                workchain_future.outputs.formation_energy_corrected.value
            )
        )    
        print(
            "Workchain Formation energy (corrected and aligned): {} eV".format(
                workchain_future.outputs.formation_energy_corrected_algned.value
            )
        )
    else:
        print('Excepted')
else:
    if workchain_future.is_excepted:
        print('Excepted')
    else:
        print('Not yet finished')
